In [ ]:
using DifferentialEquations
using ModelingToolkit
using LinearAlgebra
using Plots
using Colors
using Images
using Statistics
using Sundials
using Random
using JLD2
using DSP

In [ ]:
function setup(r)
    # Generate constants
    N = 100
    SA = 4*pi*r^2
    V = (4/3)*pi*r^3
    mem_thickness = 0.01
    n = (mem_thickness * SA) / V

    Ax = Array(Tridiagonal([1.0 for i in 1:N-1],[-2.0 for i in 1:N],[1.0 for i in 1:N-1]))
    Ax[1,end] = 1.0
    Ax[end,1] = 1.0
    dx = (r*sqrt(pi))/N
    Ax = Ax/(dx^2) # adjust for 1/microns
    Ay = copy(Ax)

    r0 = zeros(100,100,6)
    r0[:,:,1] .= 5 .*(rand.())   # Cdc42-GTPm
    #r0[50,50,1] = 500
    r0[:,:,2] .= 0.5 - mean(r0[:,:,1])*n   # Cdc42-GDPm should be 0.5
    r0[:,:,3] .= 0 #1
    r0[:,:,4] .= 10 .*(rand.())
    r0[:,:,5] .= 0 #1
    r0[:,:,6] .= 0.75
    
    # Dummy parameters used only locally in fxn but passed to specify scope, or something..
    Ayt = zeros(N,N)
    tAx = zeros(N,N)
    D42t = zeros(N,N)
    D42d = zeros(N,N)
    Dpak = zeros(N,N)
    Dgef = zeros(N,N)
    Dpakc = zeros(N,N)
    Dgefc = zeros(N,N)
    R = zeros(N,N)
    dummy = (Ayt, tAx, D42t, D42d, Dpak, Dgef, Dpakc, Dgefc, R)
    # Actual parameters
    a = .8
    b = .07
    c = 1
    d = .05
    e = .032
    f = .1
    g = .025 
    h = 0.1
    Dm = .01
    Dc = 10
    Dm2 = .1
    n = n

    p = [a, b, c, d, e, f, g, h, Dm, Dc, Dm2, n, Ax, Ay, dummy]
    return p, r0
end

In [ ]:
function negfb!(dr,r,p,t)
    a, b, c, d, e, f, g, h, Dm, Dc, Dm2, n, Ax, Ay, dummy = p
    Ayt, tAx, D42t, D42d, Dpak, Dgef, Dpakc, Dgefc, R = dummy
    # Window variables
    rhoT = @view r[:,:,1]
    rhoD = @view r[:,:,2]
    pak = @view r[:,:,3]
    gef = @view r[:,:,4]
    pakc = @view r[:,:,5]
    gefc = @view r[:,:,6]
    # Calculate diffusion
    mul!(Ayt,Ay,rhoT)
    mul!(tAx,rhoT,Ax)
    @. D42t = Dm*(Ayt + tAx)
    mul!(Ayt,Ay,rhoD)
    mul!(tAx,rhoD,Ax)
    @. D42d = Dc*(Ayt + tAx)
    mul!(Ayt,Ay,pak)
    mul!(tAx,pak,Ax)
    @. Dpak = Dm2*(Ayt + tAx)
    mul!(Ayt,Ay,gef)
    mul!(tAx,gef,Ax)
    @. Dgef = Dm*(Ayt + tAx)
    mul!(Ayt,Ay,pakc)
    mul!(tAx,pakc,Ax)
    @. Dpakc = Dc*(Ayt + tAx)
    mul!(Ayt,Ay,gefc)
    mul!(tAx,gefc,Ax)
    @. Dgefc = Dc*(Ayt + tAx)
    # Calculate reactions, add diffusion
    @. R = (a*gef^2*rhoD) - b*rhoT
    @. dr[:,:,1] = R + D42t
    @. dr[:,:,2] = -R*n + D42d
    @. dr[:,:,3] = d*rhoT*pakc - e*pak + Dpak
    @. dr[:,:,4] = f*rhoT*gefc - g*pak*gef - h*gef + Dgef 
    @. dr[:,:,5] = n*(- d*rhoT*pakc + e*pak) + Dpakc
    @. dr[:,:,6] = n*(- f*rhoT*gefc + g*pak*gef + h*gef) + Dgefc
end


In [ ]:
p, r0 = setup(radius)
min_prob = ODEProblem(simple!,r0,(0.0,800),p)
sol = solve(min_prob,CVODE_BDF(linear_solver = :GMRES),saveat=1)

In [ ]:
Random.seed!(1)
sol3 = run_pde(3)